In [83]:
if (!require(pacman)) install.packages(pacman)
require(pacman)

In [84]:
p_load(
  rio,      # Importar/exportar datos
  tidyverse,
  # Manejo de datos ordenados (incluye dplyr, ggplot2, tidyr, readr, etc.)
  skimr,    # Resúmenes rápidos de datos
  gridExtra,
  # Visualización de datos faltantes y organizar gráficos en cuadrículas
  corrplot, # Gráficos de correlación
  stargazer, # Tablas y salida para LaTeX
  MASS,
  # Cálculos posteriores a la regresión y distribuciones estadísticas
  rvest,    # Web scraping
  httr,     # Realizar solicitudes HTTP (APIs y scraping)
  boot,     # Herramientas para Bootstrap y remuestreo estadístico
  broom,    # Convertir resultados de modelos a df_geih frames ordenados
  lmtest,   # Pruebas estadísticas para modelos lineales
  fixest,   # Estimadores con efectos fijos (modelos lineales avanzados)
  sandwich,   # Estimaciones de varianza robusta
  future.apply,   # Paralelización y ejecución asincrónica
  furrr,    # Versión paralela de funciones de purrr
  memoise   # Funciones con caché para evitar cálculos repetitivos
)

In [85]:
seed <- 555

### Web Scrapping

In [86]:
urls    <- paste0('https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_',1:10,'.html') # nolint
c_geih  <- future_lapply(
        urls,# nolint
        function(url){# nolint  
                html <- read_html(url)# nolint  
                return()(html_node(html, "table") %>% html_table())# nolint  
                })# nolint  

df <- do.call(rbind, df_list)

# write.csv(df_geih, "geih.csv", row.names = FALSE) # nolint

In [87]:
df_geih <- import("https://github.com/EAriasDeAbreu/Bigdf_geihML-Group2024/raw/main/Problem_Set_1/df_geih/GEIH.Rdf_geih")

Warning message:
"The `trust` argument of `import()` should be explicit for serialization formats
as of rio 1.0.3.
i Missing `trust` will be set to FALSE by default for RData in 2.0.0.
i The deprecated feature was likely used in the rio package.
  Please report the issue at <https://github.com/gesistsa/rio/issues>."


### Limpieza

In [88]:
df_geih  <- df_geih %>%
            filter(                             # nolint
            age > 18 &                          # nolint
            ocu == 1 &                          # nolint
            !is.na(y_salary_m) &                # nolint
            !is.na(age)) %>%
            mutate(ln_w = log(y_salary_m + 1))  # nolint

In [99]:
# Crear variable dicotómica para p6580 (bonificaciones recibidas el mes pasado)
df_geih$bonif <- ifelse(df_geih$p6580 == 1, 1, ifelse(df_geih$p6580 == 2, 0, NA))

# Crear variable dicotómica para y_horasExtras_m (ingreso por horas extras)
df_geih$extra_h <- ifelse(df_geih$y_horasExtras_m > 0, 1, 0)

# Crear variable dicotómica para p7472 (ingresos por trabajo en desocupados)
df_geih$desoc <- ifelse(df_geih$p7472 == 1, 1, 0)

## Estimación sugerida

In [69]:
age_wage_profile <- lm(ln_w ~ age + I(age^2), df_geih = df_geih)

In [70]:
summary(age_wage_profile)


Call:
lm(formula = ln_w ~ age + I(age^2), data = df_geih)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7449 -0.3833 -0.1060  0.3007  3.8503 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.238e+01  7.335e-02  168.73   <2e-16 ***
age          8.509e-02  3.878e-03   21.94   <2e-16 ***
I(age^2)    -9.880e-04  4.773e-05  -20.70   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.7423 on 9782 degrees of freedom
Multiple R-squared:  0.05007,	Adjusted R-squared:  0.04988 
F-statistic: 257.8 on 2 and 9782 DF,  p-value: < 2.2e-16
